<a href="https://colab.research.google.com/github/ArunK-ML/Project---Nutrition-Paradox-A-Global-View-on-Obesity-and-Malnutrition/blob/main/Nutrition_Paradox_Streamlit_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Streamlit**

In [ ]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 27.2 MB/s eta 0:00:00


In [ ]:
%%writefile Nutrition_streamlit_app.py

# Nutrition_streamlit_app.py
import streamlit as st
import mysql.connector
import pandas as pd
import plotly.express as px

# Page config
st.set_page_config(page_title="Health Nutrition Dashboard", layout="wide")

# Add logo and title to the sidebar
st.sidebar.image("https://upload.wikimedia.org/wikipedia/commons/thumb/4/4a/Commons-emblem-health_and_safety.svg/1024px-Commons-emblem-health_and_safety.svg.png", width=100)
st.sidebar.title("🥗 Health Nutrition Dashboard")

# Database connection function
def get_db_connection():
    return mysql.connector.connect(
        host="gateway01.us-east-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2RF9f3Nuh4kLJBe.root",
        password="3nvycLptvyoI59Us",
        database="Health_Database"
    )

# Query executor
def run_query(query):
    conn = get_db_connection()
    try:
        df = pd.read_sql(query, conn)
        return df
    except Exception as e:
        st.error(f"Error executing query: {e}")
        return None
    finally:
        conn.close()

# Sidebar navigation
page = st.sidebar.radio("Navigate", ["Home", "Obesity Queries", "Malnutrition Queries", "Combined Insights", "About"])

# Page 1: Home
if page == "Home":
    st.title("📊 Nutrition Insights Explorer")
    col1, col2 = st.columns([2, 1])
    with col1:
        st.markdown("""
        ### Welcome to the Health Nutrition Dashboard
        Explore and compare **Obesity** and **Malnutrition** statistics across regions, age groups, genders, and time.
        Use the side menu to run pre-defined queries and visualize trends interactively.
        """)
    with col2:
        st.image("https://www.who.int/images/default-source/health-topics/nutrition/nutrition-graphic.jpg", width=300)

# Page 2: Obesity Queries
elif page == "Obesity Queries":
    st.header("🧍 Obesity-related Queries")
    question = st.selectbox("Select a query:", [
        "Top 5 regions with highest obesity in 2022",
        "Top 5 countries by max obesity",
        "Obesity trend in India",
        "Obesity by gender",
        "Country count by obesity level & age",
        "Countries with highest & lowest CI width",
        "Obesity by age group",
        "Consistently low obesity countries",
        "Female obesity much higher than male",
        "Global yearly average obesity"
    ])

    obesity_queries = {
        "Top 5 regions with highest obesity in 2022": "SELECT Region, AVG(Mean_Estimate) AS avg_obesity FROM obesity WHERE Year = 2022 GROUP BY Region ORDER BY avg_obesity DESC LIMIT 5;",
        "Top 5 countries by max obesity": "SELECT Country, MAX(Mean_Estimate) AS max_obesity FROM obesity GROUP BY Country ORDER BY max_obesity DESC LIMIT 5;",
        "Obesity trend in India": "SELECT Year, Mean_Estimate FROM obesity WHERE Country = 'India' ORDER BY Year;",
        "Obesity by gender": "SELECT Gender, AVG(Mean_Estimate) AS avg_obesity FROM obesity GROUP BY Gender;",
        "Country count by obesity level & age": "SELECT Country, age_group, COUNT(*) AS country_count FROM obesity GROUP BY Country, age_group;",
        "Countries with highest & lowest CI width": "SELECT Country, AVG(CI_Width) AS avg_ci_width FROM obesity GROUP BY Country ORDER BY avg_ci_width ASC LIMIT 5;",
        "Obesity by age group": "SELECT age_group, AVG(Mean_Estimate) AS avg_obesity FROM obesity GROUP BY age_group;",
        "Consistently low obesity countries": "SELECT Country, AVG(Mean_Estimate) AS avg_obesity FROM obesity GROUP BY Country ORDER BY avg_obesity ASC LIMIT 10;",
        "Female obesity much higher than male": "SELECT Year, Country, Mean_Estimate FROM obesity WHERE Gender = 'Female' AND Mean_Estimate > (SELECT MAX(Mean_Estimate) FROM obesity AS m WHERE m.Gender = 'Male' AND m.Year = obesity.Year AND m.Country = obesity.Country);",
        "Global yearly average obesity": "SELECT Year, AVG(Mean_Estimate) AS global_avg_obesity FROM obesity GROUP BY Year ORDER BY Year;"
    }

    query = obesity_queries[question]
    if st.button("Run Query"):
        df = run_query(query)
        if df is not None:
            st.dataframe(df, use_container_width=True)
            numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
            if not df.empty and len(numeric_cols) >= 1:
                col = st.selectbox("Select numeric column to plot:", numeric_cols)
                fig = px.bar(df, x=df.columns[0], y=col, title=question)
                st.plotly_chart(fig, use_container_width=True)

# Page 3: Malnutrition Queries
elif page == "Malnutrition Queries":
    st.header("🍎 Malnutrition-related Queries")
    question = st.selectbox("Select a query:", [
        "Avg. malnutrition by age group",
        "Top 5 countries with highest malnutrition",
        "Malnutrition trend in Africa",
        "Gender-based average malnutrition",
        "CI Width by malnutrition level & age",
        "Malnutrition trend in India, Nigeria, Brazil",
        "Regions with lowest malnutrition",
        "Countries with increasing malnutrition",
        "Yearly min/max comparison",
        "High CI width cases"
    ])

    malnutrition_queries = {
        "Avg. malnutrition by age group": "SELECT age_group, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition GROUP BY age_group;",
        "Top 5 countries with highest malnutrition": "SELECT Country, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition GROUP BY Country ORDER BY avg_malnutrition DESC LIMIT 5;",
        "Malnutrition trend in Africa": "SELECT Year, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition WHERE Region = 'Africa' GROUP BY Year;",
        "Gender-based average malnutrition": "SELECT Gender, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition GROUP BY Gender;",
        "CI Width by malnutrition level & age": "SELECT age_group, AVG(CI_Width) AS avg_ci_width FROM malnutrition GROUP BY age_group;",
        "Malnutrition trend in India, Nigeria, Brazil": "SELECT Year, Country, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition WHERE Country IN ('India', 'Nigeria', 'Brazil') GROUP BY Year, Country;",
        "Regions with lowest malnutrition": "SELECT Region, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition GROUP BY Region ORDER BY avg_malnutrition ASC LIMIT 5;",
        "Countries with increasing malnutrition": "SELECT Country, MIN(Mean_Estimate), MAX(Mean_Estimate) FROM malnutrition GROUP BY Country HAVING MIN(Mean_Estimate) < MAX(Mean_Estimate);",
        "Yearly min/max comparison": "SELECT Year, MIN(Mean_Estimate), MAX(Mean_Estimate) FROM malnutrition GROUP BY Year;",
        "High CI width cases": "SELECT * FROM malnutrition WHERE CI_Width > 5 ORDER BY CI_Width DESC;"
    }

    query = malnutrition_queries[question]
    if st.button("Run Query"):
        df = run_query(query)
        if df is not None:
            st.dataframe(df, use_container_width=True)
            numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
            if not df.empty and len(numeric_cols) >= 1:
                col = st.selectbox("Select numeric column to plot:", numeric_cols)
                fig = px.line(df, x=df.columns[0], y=col, title=question)
                st.plotly_chart(fig, use_container_width=True)

# Page 4: Obesity & Malnutrition Queries
elif page == "Obesity & Malnutrition Queries":
    st.header("🍎 Obesity & Malnutrition-related Queries")
    question = st.selectbox("Select a query:", [
      "1. Obesity vs malnutrition comparison by country(any 5 countries)",
      "2. Gender-based disparity in both obesity and malnutrition",
      "3. Region-wise avg estimates side-by-side(Africa and America)",
      "4. Countries with obesity up & malnutrition down",
      "5. Age-wise trend analysis"
      ])

Combined_queries = {
      "SELECT o.Country AVG(o.Mean_Estimate) AS avg_obesity AVG(m.Mean_Estimate) AS avg_malnutrition FROM obesity o JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year WHERE o.Country IN ('India', 'USA', 'Nigeria', 'Brazil', 'China') GROUP BY o.Country;",
      "SELECT o.Gender,AVG(o.Mean_Estimate) AS avg_obesity,AVG(m.Mean_Estimate) AS avg_malnutrition FROM obesity o JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year GROUP BY o.Gender;",
      "SELECT o.Region, o.Country,AVG(o.Mean_Estimate) AS avg_obesity,AVG(m.Mean_Estimate) AS avg_malnutrition FROM obesity o JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year GROUP BY o.Region, o.Country;",
      "SELECT o.Country, o.Year, o.Mean_Estimate AS obesity_estimate, m.Mean_Estimate AS malnutrition_estimate FROM obesity o JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year",
      "SELECT o.age_group, o.Year, o.Mean_Estimate AS obesity_estimate, m.Mean_Estimate AS malnutrition_estimate FROM obesity o JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year"
}
      query = Combined_queries[question]
      if st.button("Run Query"):
          df = run_query(query)
          if df is not None:
              st.dataframe(df, use_container_width=True)
              numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
              if not df.empty and len(numeric_cols) >= 1:
                  col = st.selectbox("Select numeric column to plot:", numeric_cols)
                  fig = px.line(df, x=df.columns[0], y=col, title=question)
                  st.plotly_chart(fig, use_container_width=True)

# Page 5: Combined Insights
elif page == "Combined Insights":
    st.header("📈 Obesity vs Malnutrition Comparison")
    query = """
    SELECT o.Country, AVG(o.Mean_Estimate) AS avg_obesity, AVG(m.Mean_Estimate) AS avg_malnutrition
    FROM obesity o
    JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year
    GROUP BY o.Country ORDER BY o.Country;
    """
    df = run_query(query)
    if df is not None:
        st.dataframe(df, use_container_width=True)
        fig = px.scatter(df, x="avg_obesity", y="avg_malnutrition", text="Country", title="Obesity vs Malnutrition by Country")
        st.plotly_chart(fig, use_container_width=True)

# Page 5: About
elif page == "About":
    st.header("📚 About This App")
    st.write("This dashboard is built using **Streamlit**, **Plotly**, and **MySQL**. It provides interactive visualizations and SQL-powered insights into global **obesity** and **malnutrition** trends.")
    st.write("**Data Source**: Health_Database (from TiDB Cloud)")
    st.markdown("---")
    st.write("Developed by **Arun Kumar**")
    st.caption("Thank you for visiting!")

st.markdown("---")
st.caption("Developed by Arun Kumar | Powered by GUVI Data")


Overwriting Nutrition_streamlit_app.py


In [ ]:
!pip install streamlit streamlit_option_menu  # installing streamlit and streamlit_option_menu packages

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.5 MB/s eta 0:00:00


In [ ]:
!wget -q -O - ipv4.icanhazip.com   # this command will generate a password for you(copy that)

34.46.5.48


In [ ]:
!streamlit run Nutrition_streamlit_app.py & npx localtunnel --port 8501  # replace the file name accordingly

⠙⠹⠸⠼⠴

⠦⠧⠇⠏⠋⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.46.5.48:8501

⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼your url is: https://tangy-plums-boil.loca.lt
/content/Nutrition_streamlit_app.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/content/Nutrition_streamlit_app.py:29: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/content/Nutrition_streamlit_app.py:29: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

In [ ]:
%%writefile Nu_streamlit_app.py
import streamlit as st
import mysql.connector
import pandas as pd
import plotly.express as px

# Set Streamlit page config
st.set_page_config(page_title="🌍 Nutrition Dashboard", layout="wide")

# Establish MySQL connection
@st.cache_resource

def get_connection():
    return mysql.connector.connect(
        host="gateway01.us-east-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2RF9f3Nuh4kLJBe.root",
        password="3nvycLptvyoI59Us",
        database="Health_Database"
    )

# Fetch data from database
@st.cache_data

def load_data():
    conn = get_connection()
    df_obesity = pd.read_sql("SELECT * FROM obesity", conn)
    df_malnutrition = pd.read_sql("SELECT * FROM malnutrition", conn)
    conn.close()
    return df_obesity, df_malnutrition

# Load data
obesity_data, malnutrition_data = load_data()

# Sidebar navigation
page = st.sidebar.radio("Select Page", [
    "Home",
    "Explore Obesity Data",
    "Explore Malnutrition Data",
    "Obesity & Malnutrition Queries"
])

# Page 1: Home
if page == "Home":
    st.title("🌍 Nutrition Dashboard")
    st.markdown("""
    Welcome to the interactive dashboard comparing global **Obesity** and **Malnutrition** levels using WHO data.

    Use the sidebar to explore:
    - Obesity patterns across countries, genders, and age groups
    - Malnutrition levels over time and region
    - Combined analytics of both indicators
    """)

# Page 2: Obesity Exploration
elif page == "Explore Obesity Data":
    st.header("📊 Explore Obesity Data")

    selected_region = st.selectbox("Select Region:", obesity_data["Region"].unique())
    filtered_data = obesity_data[obesity_data["Region"] == selected_region]

    fig = px.line(filtered_data, x="Year", y="Mean_Estimate", color="Country",
                  title=f"Obesity Trend in {selected_region}")
    st.plotly_chart(fig, use_container_width=True)

# Page 3: Malnutrition Exploration
elif page == "Explore Malnutrition Data":
    st.header("📉 Explore Malnutrition Data")

    selected_country = st.selectbox("Select Country:", malnutrition_data["Country"].unique())
    filtered_data = malnutrition_data[malnutrition_data["Country"] == selected_country]

    fig = px.line(filtered_data, x="Year", y="Mean_Estimate", color="Gender",
                  title=f"Malnutrition Trend in {selected_country}")
    st.plotly_chart(fig, use_container_width=True)

# Page 4: Obesity & Malnutrition Queries
elif page == "Obesity & Malnutrition Queries":
    st.header("🍎 Obesity & Malnutrition-related Queries")

    # Select query from dropdown
    question = st.selectbox("Select a query:", [
        "1. Obesity vs malnutrition comparison by country(any 5 countries)",
        "2. Gender-based disparity in both obesity and malnutrition",
        "3. Region-wise avg estimates side-by-side(Africa and America)",
        "4. Countries with obesity up & malnutrition down",
        "5. Age-wise trend analysis"
    ])

    # Define combined SQL queries dictionary
    combined_queries = {
        "1. Obesity vs malnutrition comparison by country(any 5 countries)": """
            SELECT o.Country,
                   AVG(o.Mean_Estimate) AS avg_obesity,
                   AVG(m.Mean_Estimate) AS avg_malnutrition
            FROM obesity o
            JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year
            WHERE o.Country IN ('India', 'USA', 'Nigeria', 'Brazil', 'China')
            GROUP BY o.Country;
        """,
        "2. Gender-based disparity in both obesity and malnutrition": """
            SELECT o.Gender,
                   AVG(o.Mean_Estimate) AS avg_obesity,
                   AVG(m.Mean_Estimate) AS avg_malnutrition
            FROM obesity o
            JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year AND o.Gender = m.Gender
            GROUP BY o.Gender;
        """,
        "3. Region-wise avg estimates side-by-side(Africa and America)": """
            SELECT o.Region,
                   AVG(o.Mean_Estimate) AS avg_obesity,
                   AVG(m.Mean_Estimate) AS avg_malnutrition
            FROM obesity o
            JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year
            WHERE o.Region IN ('Africa', 'Americas')
            GROUP BY o.Region;
        """,
        "4. Countries with obesity up & malnutrition down": """
            SELECT o.Country, o.Year,
                   o.Mean_Estimate AS obesity_estimate,
                   m.Mean_Estimate AS malnutrition_estimate
            FROM obesity o
            JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year
            WHERE o.Mean_Estimate > m.Mean_Estimate;
        """,
        "5. Age-wise trend analysis": """
            SELECT o.age_group, o.Year,
                   AVG(o.Mean_Estimate) AS obesity_estimate,
                   AVG(m.Mean_Estimate) AS malnutrition_estimate
            FROM obesity o
            JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year AND o.age_group = m.age_group
            GROUP BY o.age_group, o.Year;
        """
    }

    # Execute query
    conn = get_connection()
    df_result = pd.read_sql(combined_queries[question], conn)
    conn.close()

    # Show results
    st.dataframe(df_result)

    # Plot if time-based or group-based data
    if "Year" in df_result.columns:
        if "Country" in df_result.columns:
            fig = px.line(df_result, x="Year", y=df_result.columns[-2], color="Country",
                          title="Trend Over Time")
        elif "age_group" in df_result.columns:
            fig = px.line(df_result, x="Year", y="obesity_estimate", color="age_group",
                          title="Age-wise Obesity Trends")
        else:
            fig = px.line(df_result, x="Year", y=df_result.columns[-1], title="Trend")
        st.plotly_chart(fig, use_container_width=True)


Writing Nu_streamlit_app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.46.5.48


In [ ]:
!streamlit run Nu_streamlit_app.py & npx localtunnel --port 8501  # replace the file name accordingly

⠙

⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.46.5.48:8501

⠴⠦⠧⠇⠏⠋⠙⠹your url is: https://hungry-eels-carry.loca.lt
/content/Nu_streamlit_app.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_obesity = pd.read_sql("SELECT * FROM obesity", conn)
/content/Nu_streamlit_app.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_malnutrition = pd.read_sql("SELECT * FROM malnutrition", conn)
/content/Nu_streamlit_app.py:137: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBA

In [ ]:
%%writefile Nutrition_streamlit_app.py

# Nutrition_streamlit_app.py
import streamlit as st
import mysql.connector
import pandas as pd
import plotly.express as px

# Page config
st.set_page_config(page_title="Health Nutrition Dashboard", layout="wide")

# Add logo and title to the sidebar
st.sidebar.image("https://medicircle.in/uploads/2021/january2021/how-important-is-nutrition-for-health.jpg", width=100)
st.sidebar.title("🥗 Health Nutrition Dashboard")

# Database connection function
def get_db_connection():
    return mysql.connector.connect(
        host="gateway01.us-east-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2RF9f3Nuh4kLJBe.root",
        password="3nvycLptvyoI59Us",
        database="Health_Database"
    )

# Query executor
def run_query(query):
    conn = get_db_connection()
    try:
        df = pd.read_sql(query, conn)
        return df
    except Exception as e:
        st.error(f"Error executing query: {e}")
        return None
    finally:
        conn.close()

# Sidebar navigation
page = st.sidebar.radio("Navigate", ["Home", "Queries", "Combined Insights", "About"])

Obesity Queries = [
        "Top 5 regions with highest obesity in 2022",
        "Top 5 countries by max obesity",
        "Obesity trend in India",
        "Obesity by gender",
        "Country count by obesity level & age",
        "Countries with highest & lowest CI width",
        "Obesity by age group",
        "Consistently low obesity countries",
        "Female obesity much higher than male",
        "Global yearly average obesity"
    ]

obesity_queries_ans = [
        "SELECT Region, AVG(Mean_Estimate) AS avg_obesity FROM obesity WHERE Year = 2022 GROUP BY Region ORDER BY avg_obesity DESC LIMIT 5;",
        "SELECT Country, MAX(Mean_Estimate) AS max_obesity FROM obesity GROUP BY Country ORDER BY max_obesity DESC LIMIT 5;",
        "SELECT Year, Mean_Estimate FROM obesity WHERE Country = 'India' ORDER BY Year;",
        "SELECT Gender, AVG(Mean_Estimate) AS avg_obesity FROM obesity GROUP BY Gender;",
        "SELECT Country, age_group, COUNT(*) AS country_count FROM obesity GROUP BY Country, age_group;",
        "SELECT Country, AVG(CI_Width) AS avg_ci_width FROM obesity GROUP BY Country ORDER BY avg_ci_width ASC LIMIT 5;",
        "SELECT age_group, AVG(Mean_Estimate) AS avg_obesity FROM obesity GROUP BY age_group;",
        "SELECT Country, AVG(Mean_Estimate) AS avg_obesity FROM obesity GROUP BY Country ORDER BY avg_obesity ASC LIMIT 10;",
        "SELECT Year, Country, Mean_Estimate FROM obesity WHERE Gender = 'Female' AND Mean_Estimate > (SELECT MAX(Mean_Estimate) FROM obesity AS m WHERE m.Gender = 'Male' AND m.Year = obesity.Year AND m.Country = obesity.Country);",
        "SELECT Year, AVG(Mean_Estimate) AS global_avg_obesity FROM obesity GROUP BY Year ORDER BY Year;"
    ]

Malnutrition_Queries = [
        "Avg. malnutrition by age group",
        "Top 5 countries with highest malnutrition",
        "Malnutrition trend in Africa",
        "Gender-based average malnutrition",
        "CI Width by malnutrition level & age",
        "Malnutrition trend in India, Nigeria, Brazil",
        "Regions with lowest malnutrition",
        "Countries with increasing malnutrition",
        "Yearly min/max comparison",
        "High CI width cases"
    ]
Malnutrition_Queries_ans = [
        "SELECT age_group, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition GROUP BY age_group;",
        "SELECT Country, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition GROUP BY Country ORDER BY avg_malnutrition DESC LIMIT 5;",
        "SELECT Year, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition WHERE Region = 'Africa' GROUP BY Year;",
        "SELECT Gender, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition GROUP BY Gender;",
        "SELECT age_group, AVG(CI_Width) AS avg_ci_width FROM malnutrition GROUP BY age_group;",
        "SELECT Year, Country, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition WHERE Country IN ('India', 'Nigeria', 'Brazil') GROUP BY Year, Country;",
        "SELECT Region, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition GROUP BY Region ORDER BY avg_malnutrition ASC LIMIT 5;",
        "SELECT Country, MIN(Mean_Estimate), MAX(Mean_Estimate) FROM malnutrition GROUP BY Country HAVING MIN(Mean_Estimate) < MAX(Mean_Estimate);",
        "SELECT Year, MIN(Mean_Estimate), MAX(Mean_Estimate) FROM malnutrition GROUP BY Year;",
        "SELECT * FROM malnutrition WHERE CI_Width > 5 ORDER BY CI_Width DESC;"
]


# Page 1: Home
if page == "Home":
    st.title("📊 Nutrition Insights Explorer")
    col1, col2 = st.columns([2, 1])
    with col1:
        st.markdown("""
        ### Welcome to the Health Nutrition Dashboard
        Explore and compare **Obesity** and **Malnutrition** statistics across regions, age groups, genders, and time.
        Use the side menu to run pre-defined queries and visualize trends interactively.
        """)
    with col2:
        st.image("https://www.who.int/images/default-source/health-topics/nutrition/nutrition-graphic.jpg", width=300)







# Page 2: Obesity Queries
elif page == "Obesity Queries":
    st.header("🧍 Obesity-related Queries")
    question = st.selectbox("Select a query:", [
        "Top 5 regions with highest obesity in 2022",
        "Top 5 countries by max obesity",
        "Obesity trend in India",
        "Obesity by gender",
        "Country count by obesity level & age",
        "Countries with highest & lowest CI width",
        "Obesity by age group",
        "Consistently low obesity countries",
        "Female obesity much higher than male",
        "Global yearly average obesity"
    ])

    obesity_queries = {
        "Top 5 regions with highest obesity in 2022": "SELECT Region, AVG(Mean_Estimate) AS avg_obesity FROM obesity WHERE Year = 2022 GROUP BY Region ORDER BY avg_obesity DESC LIMIT 5;",
        "Top 5 countries by max obesity": "SELECT Country, MAX(Mean_Estimate) AS max_obesity FROM obesity GROUP BY Country ORDER BY max_obesity DESC LIMIT 5;",
        "Obesity trend in India": "SELECT Year, Mean_Estimate FROM obesity WHERE Country = 'India' ORDER BY Year;",
        "Obesity by gender": "SELECT Gender, AVG(Mean_Estimate) AS avg_obesity FROM obesity GROUP BY Gender;",
        "Country count by obesity level & age": "SELECT Country, age_group, COUNT(*) AS country_count FROM obesity GROUP BY Country, age_group;",
        "Countries with highest & lowest CI width": "SELECT Country, AVG(CI_Width) AS avg_ci_width FROM obesity GROUP BY Country ORDER BY avg_ci_width ASC LIMIT 5;",
        "Obesity by age group": "SELECT age_group, AVG(Mean_Estimate) AS avg_obesity FROM obesity GROUP BY age_group;",
        "Consistently low obesity countries": "SELECT Country, AVG(Mean_Estimate) AS avg_obesity FROM obesity GROUP BY Country ORDER BY avg_obesity ASC LIMIT 10;",
        "Female obesity much higher than male": "SELECT Year, Country, Mean_Estimate FROM obesity WHERE Gender = 'Female' AND Mean_Estimate > (SELECT MAX(Mean_Estimate) FROM obesity AS m WHERE m.Gender = 'Male' AND m.Year = obesity.Year AND m.Country = obesity.Country);",
        "Global yearly average obesity": "SELECT Year, AVG(Mean_Estimate) AS global_avg_obesity FROM obesity GROUP BY Year ORDER BY Year;"
    }

    query = obesity_queries[question]
    if st.button("Run Query"):
        df = run_query(query)
        if df is not None:
            st.dataframe(df, use_container_width=True)
            numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
            if not df.empty and len(numeric_cols) >= 1:
                col = st.selectbox("Select numeric column to plot:", numeric_cols)
                fig = px.bar(df, x=df.columns[0], y=col, title=question)
                st.plotly_chart(fig, use_container_width=True)

# Page 3: Malnutrition Queries
elif page == "Malnutrition Queries":
    st.header("🍎 Malnutrition-related Queries")
    question = st.selectbox("Select a query:", [
        "Avg. malnutrition by age group",
        "Top 5 countries with highest malnutrition",
        "Malnutrition trend in Africa",
        "Gender-based average malnutrition",
        "CI Width by malnutrition level & age",
        "Malnutrition trend in India, Nigeria, Brazil",
        "Regions with lowest malnutrition",
        "Countries with increasing malnutrition",
        "Yearly min/max comparison",
        "High CI width cases"
    ])

    malnutrition_queries = {
        "Avg. malnutrition by age group": "SELECT age_group, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition GROUP BY age_group;",
        "Top 5 countries with highest malnutrition": "SELECT Country, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition GROUP BY Country ORDER BY avg_malnutrition DESC LIMIT 5;",
        "Malnutrition trend in Africa": "SELECT Year, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition WHERE Region = 'Africa' GROUP BY Year;",
        "Gender-based average malnutrition": "SELECT Gender, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition GROUP BY Gender;",
        "CI Width by malnutrition level & age": "SELECT age_group, AVG(CI_Width) AS avg_ci_width FROM malnutrition GROUP BY age_group;",
        "Malnutrition trend in India, Nigeria, Brazil": "SELECT Year, Country, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition WHERE Country IN ('India', 'Nigeria', 'Brazil') GROUP BY Year, Country;",
        "Regions with lowest malnutrition": "SELECT Region, AVG(Mean_Estimate) AS avg_malnutrition FROM malnutrition GROUP BY Region ORDER BY avg_malnutrition ASC LIMIT 5;",
        "Countries with increasing malnutrition": "SELECT Country, MIN(Mean_Estimate), MAX(Mean_Estimate) FROM malnutrition GROUP BY Country HAVING MIN(Mean_Estimate) < MAX(Mean_Estimate);",
        "Yearly min/max comparison": "SELECT Year, MIN(Mean_Estimate), MAX(Mean_Estimate) FROM malnutrition GROUP BY Year;",
        "High CI width cases": "SELECT * FROM malnutrition WHERE CI_Width > 5 ORDER BY CI_Width DESC;"
    }

    query = malnutrition_queries[question]
    if st.button("Run Query"):
        df = run_query(query)
        if df is not None:
            st.dataframe(df, use_container_width=True)
            numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
            if not df.empty and len(numeric_cols) >= 1:
                col = st.selectbox("Select numeric column to plot:", numeric_cols)
                fig = px.line(df, x=df.columns[0], y=col, title=question)
                st.plotly_chart(fig, use_container_width=True)

# Page 4: Obesity & Malnutrition Queries
elif page == "Obesity & Malnutrition Queries":
    st.header("🍎 Obesity & Malnutrition-related Queries")
    question = st.selectbox("Select a query:", [
      "1. Obesity vs malnutrition comparison by country(any 5 countries)",
      "2. Gender-based disparity in both obesity and malnutrition",
      "3. Region-wise avg estimates side-by-side(Africa and America)",
      "4. Countries with obesity up & malnutrition down",
      "5. Age-wise trend analysis"
      ])

Combined_queries = {
      "SELECT o.Country AVG(o.Mean_Estimate) AS avg_obesity AVG(m.Mean_Estimate) AS avg_malnutrition FROM obesity o JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year WHERE o.Country IN ('India', 'USA', 'Nigeria', 'Brazil', 'China') GROUP BY o.Country;",
      "SELECT o.Gender,AVG(o.Mean_Estimate) AS avg_obesity,AVG(m.Mean_Estimate) AS avg_malnutrition FROM obesity o JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year GROUP BY o.Gender;",
      "SELECT o.Region, o.Country,AVG(o.Mean_Estimate) AS avg_obesity,AVG(m.Mean_Estimate) AS avg_malnutrition FROM obesity o JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year GROUP BY o.Region, o.Country;",
      "SELECT o.Country, o.Year, o.Mean_Estimate AS obesity_estimate, m.Mean_Estimate AS malnutrition_estimate FROM obesity o JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year",
      "SELECT o.age_group, o.Year, o.Mean_Estimate AS obesity_estimate, m.Mean_Estimate AS malnutrition_estimate FROM obesity o JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year"
}
      query = Combined_queries[question]
      if st.button("Run Query"):
          df = run_query(query)
          if df is not None:
              st.dataframe(df, use_container_width=True)
              numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
              if not df.empty and len(numeric_cols) >= 1:
                  col = st.selectbox("Select numeric column to plot:", numeric_cols)
                  fig = px.line(df, x=df.columns[0], y=col, title=question)
                  st.plotly_chart(fig, use_container_width=True)

# Page 5: Combined Insights
elif page == "Combined Insights":
    st.header("📈 Obesity vs Malnutrition Comparison")
    query = """
    SELECT o.Country, AVG(o.Mean_Estimate) AS avg_obesity, AVG(m.Mean_Estimate) AS avg_malnutrition
    FROM obesity o
    JOIN malnutrition m ON o.Country = m.Country AND o.Year = m.Year
    GROUP BY o.Country ORDER BY o.Country;
    """
    df = run_query(query)
    if df is not None:
        st.dataframe(df, use_container_width=True)
        fig = px.scatter(df, x="avg_obesity", y="avg_malnutrition", text="Country", title="Obesity vs Malnutrition by Country")
        st.plotly_chart(fig, use_container_width=True)

# Page 5: About
elif page == "About":
    st.header("📚 About This App")
    st.write("This dashboard is built using **Streamlit**, **Plotly**, and **MySQL**. It provides interactive visualizations and SQL-powered insights into global **obesity** and **malnutrition** trends.")
    st.write("**Data Source**: Health_Database (from TiDB Cloud)")
    st.markdown("---")
    st.write("Developed by **Arun Kumar**")
    st.caption("Thank you for visiting!")

st.markdown("---")
st.caption("Developed by Arun Kumar | Powered by GUVI Data")
